<a href="https://www.kaggle.com/code/xinkaichen97/capstone-project-macroforge-agent?scriptVersionId=283192209" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# MacroForge Agent: Recipe Adaptation Engine for Everyone

A multi-agent system using Google ADK that adapts recipes, suggests substitutions, and generates macro-optimized shopping lists.

## Overview

### Features
- Generate recipes based on calorie and protein goals
- Suggest ingredient substitutions
- Create shopping lists
- Optimize for budget
- Meal prep scheduling
- Web search for recipes and prices
- Accurate macro calculations

### Architecture

#### How It Works

```
User Input
    ↓
Runner/InMemoryRunner
    ↓
MacroForge Agent - Orchestrator
    ↓
┌────────────────────────────────────────┐
│  AgentTools (wrapped specialists)      │
├────────────────────────────────────────┤
│  RecipeGenerator (+ google_search)     │
│  MacroCalculator (+ custom tools)      │
│  SubstitutionExpert                    │
│  ShoppingListAgent (+ google_search)   │
└────────────────────────────────────────┘
    ↓
Results → Orchestrator → User
```

#### Components

1. **Custom Tools**
   - `calculate_recipe_macros` - Precise macro calculations from ingredients
   - `suggest_macro_adjustments` - Recommendations to hit targets

2. **Built-in Tools**
   - `google_search` - Web search for recipes, prices, tips

3. **Specialized Agents**
   - `RecipeGenerator` - Creates recipes + google_search
   - `MacroCalculator` - Calculates/validates macros
   - `SubstitutionExpert` - Suggests alternatives
   - `ShoppingListAgent` - Shopping + prices + google_search

4. **Orchestrator** - Routes and coordinates all agents

5. **InMemoryRunner** - Executes workflows

### Key Features

- **Retry Logic**: All agents retry on 429, 500, 503, 504 errors (5 attempts)
- **Web Search**: RecipeGenerator and ShoppingListAgent can search online
- **Precise Macros**: Custom calculator for accurate nutrition data
- **Smart Routing**: Orchestrator chains agents for complete workflows
- **Clean Design**: No wrapper functions - agents work from instructions + tools

## Setup

### Add API key and import core ADK components

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✓ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✓ Gemini API key setup complete.


In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, google_search
from google.genai import types

print("✓ ADK components imported successfully.")

✓ ADK components imported successfully.


### Session and Memories

In [3]:
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import preload_memory
from google.adk.tools.preload_memory_tool import PreloadMemoryTool
from google.adk.runners import Runner


APP_NAME = "MacroForgeApp"
USER_ID = "demo_user"


async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )


print("✓ Helper functions defined.")

✓ Helper functions defined.


## Define Custom Tools

In [4]:
# Comprehensive nutrition database (simplified for demo)
NUTRITION_DB = {
    # Proteins (per 100g)
    'chicken breast': {'calories': 165, 'protein': 31, 'carbs': 0, 'fat': 3.6},
    'chicken': {'calories': 165, 'protein': 31, 'carbs': 0, 'fat': 3.6},
    'ground beef': {'calories': 250, 'protein': 26, 'carbs': 0, 'fat': 15},
    'beef': {'calories': 250, 'protein': 26, 'carbs': 0, 'fat': 15},
    'salmon': {'calories': 208, 'protein': 20, 'carbs': 0, 'fat': 13},
    'tuna': {'calories': 130, 'protein': 28, 'carbs': 0, 'fat': 1},
    'eggs': {'calories': 155, 'protein': 13, 'carbs': 1.1, 'fat': 11},
    'egg whites': {'calories': 52, 'protein': 11, 'carbs': 0.7, 'fat': 0.2},
    'greek yogurt': {'calories': 59, 'protein': 10, 'carbs': 3.6, 'fat': 0.4},
    'yogurt': {'calories': 59, 'protein': 10, 'carbs': 3.6, 'fat': 0.4},
    'cottage cheese': {'calories': 98, 'protein': 11, 'carbs': 3.4, 'fat': 4.3},
    'tofu': {'calories': 76, 'protein': 8, 'carbs': 1.9, 'fat': 4.8},
    'tempeh': {'calories': 193, 'protein': 19, 'carbs': 9, 'fat': 11},
    'whey protein': {'calories': 400, 'protein': 80, 'carbs': 10, 'fat': 5},
    'protein powder': {'calories': 400, 'protein': 80, 'carbs': 10, 'fat': 5},
    'turkey': {'calories': 135, 'protein': 30, 'carbs': 0, 'fat': 1},
    'shrimp': {'calories': 99, 'protein': 24, 'carbs': 0.2, 'fat': 0.3},
    
    # Carbs (per 100g)
    'white rice': {'calories': 130, 'protein': 2.7, 'carbs': 28, 'fat': 0.3},
    'rice': {'calories': 130, 'protein': 2.7, 'carbs': 28, 'fat': 0.3},
    'brown rice': {'calories': 111, 'protein': 2.6, 'carbs': 23, 'fat': 0.9},
    'sweet potato': {'calories': 86, 'protein': 1.6, 'carbs': 20, 'fat': 0.1},
    'potato': {'calories': 77, 'protein': 2, 'carbs': 17, 'fat': 0.1},
    'oats': {'calories': 389, 'protein': 17, 'carbs': 66, 'fat': 7},
    'oatmeal': {'calories': 389, 'protein': 17, 'carbs': 66, 'fat': 7},
    'pasta': {'calories': 131, 'protein': 5, 'carbs': 25, 'fat': 1.1},
    'bread': {'calories': 265, 'protein': 9, 'carbs': 49, 'fat': 3.2},
    'quinoa': {'calories': 120, 'protein': 4.4, 'carbs': 21, 'fat': 1.9},
    'whole wheat bread': {'calories': 247, 'protein': 13, 'carbs': 41, 'fat': 3.4},
    'bagel': {'calories': 275, 'protein': 11, 'carbs': 53, 'fat': 1.5},
    'tortilla': {'calories': 218, 'protein': 6, 'carbs': 36, 'fat': 5},
    
    # Vegetables (per 100g)
    'broccoli': {'calories': 34, 'protein': 2.8, 'carbs': 7, 'fat': 0.4},
    'spinach': {'calories': 23, 'protein': 2.9, 'carbs': 3.6, 'fat': 0.4},
    'kale': {'calories': 35, 'protein': 2.9, 'carbs': 4.4, 'fat': 1.5},
    'asparagus': {'calories': 20, 'protein': 2.2, 'carbs': 3.9, 'fat': 0.1},
    'bell pepper': {'calories': 31, 'protein': 1, 'carbs': 6, 'fat': 0.3},
    'tomato': {'calories': 18, 'protein': 0.9, 'carbs': 3.9, 'fat': 0.2},
    'cucumber': {'calories': 15, 'protein': 0.7, 'carbs': 3.6, 'fat': 0.1},
    'avocado': {'calories': 160, 'protein': 2, 'carbs': 9, 'fat': 15},
    'lettuce': {'calories': 15, 'protein': 1.4, 'carbs': 2.9, 'fat': 0.1},
    'mushroom': {'calories': 22, 'protein': 3.1, 'carbs': 3.3, 'fat': 0.3},
    'onion': {'calories': 40, 'protein': 1.1, 'carbs': 9.3, 'fat': 0.1},
    'garlic': {'calories': 149, 'protein': 6.4, 'carbs': 33, 'fat': 0.5},
    
    # Fruits (per 100g)
    'banana': {'calories': 89, 'protein': 1.1, 'carbs': 23, 'fat': 0.3},
    'apple': {'calories': 52, 'protein': 0.3, 'carbs': 14, 'fat': 0.2},
    'berries': {'calories': 57, 'protein': 0.7, 'carbs': 14, 'fat': 0.3},
    'strawberry': {'calories': 32, 'protein': 0.7, 'carbs': 7.7, 'fat': 0.3},
    'blueberry': {'calories': 57, 'protein': 0.7, 'carbs': 14, 'fat': 0.3},
    'orange': {'calories': 47, 'protein': 0.9, 'carbs': 12, 'fat': 0.1},
    
    # Fats (per 100g)
    'olive oil': {'calories': 884, 'protein': 0, 'carbs': 0, 'fat': 100},
    'oil': {'calories': 884, 'protein': 0, 'carbs': 0, 'fat': 100},
    'butter': {'calories': 717, 'protein': 0.9, 'carbs': 0.1, 'fat': 81},
    'peanut butter': {'calories': 588, 'protein': 25, 'carbs': 20, 'fat': 50},
    'almonds': {'calories': 579, 'protein': 21, 'carbs': 22, 'fat': 50},
    'walnuts': {'calories': 654, 'protein': 15, 'carbs': 14, 'fat': 65},
    'cheese': {'calories': 402, 'protein': 25, 'carbs': 1.3, 'fat': 33},
    'cheddar': {'calories': 402, 'protein': 25, 'carbs': 1.3, 'fat': 33},
    'mozzarella': {'calories': 280, 'protein': 28, 'carbs': 2.2, 'fat': 17},
}

def calculate_recipe_macros(ingredients_text: str) -> dict:
    """
    Calculate total macros for a recipe based on ingredients.
    
    Args:
        ingredients_text: Recipe ingredients as text (e.g., "200g chicken breast, 150g rice")
    
    Returns:
        Dictionary with total calories, protein, carbs, fat
    """
    import re
    
    total_calories = 0
    total_protein = 0
    total_carbs = 0
    total_fat = 0
    
    parsed_ingredients = []
    missing_ingredients = []
    
    # Parse ingredients (simple regex for demo)
    lines = ingredients_text.lower().split('\n')
    for line in lines:
        if not line.strip():
            continue
            
        # Match patterns like "200g chicken" or "1 cup rice"
        match = re.search(r'(\d+\.?\d*)\s*(g|grams?|oz|ounces?|cup|cups?|tbsp?|tablespoons?|tsp|teaspoons?)\s*(.+)', line)
        if match:
            amount = float(match.group(1))
            unit = match.group(2)
            ingredient = match.group(3).strip(' -,.!?()[]')
            
            # Convert to grams
            if 'cup' in unit:
                amount = amount * 200  # rough conversion
            elif 'oz' in unit or 'ounce' in unit:
                amount = amount * 28.35
            elif 'tbsp' in unit or 'tablespoon' in unit:
                amount = amount * 15
            elif 'tsp' in unit or 'teaspoon' in unit:
                amount = amount * 5
            
            # Find matching ingredient in database
            found = False
            for db_ingredient, nutrition in NUTRITION_DB.items():
                if db_ingredient in ingredient or ingredient in db_ingredient:
                    # Scale nutrition to actual amount (DB is per 100g)
                    factor = amount / 100
                    calories = nutrition['calories'] * factor
                    protein = nutrition['protein'] * factor
                    carbs = nutrition['carbs'] * factor
                    fat = nutrition['fat'] * factor
                    
                    total_calories += calories
                    total_protein += protein
                    total_carbs += carbs
                    total_fat += fat
                    
                    parsed_ingredients.append({
                        'ingredient': db_ingredient,
                        'amount_g': round(amount, 1),
                        'calories': round(calories, 1),
                        'protein': round(protein, 1),
                        'carbs': round(carbs, 1),
                        'fat': round(fat, 1)
                    })
                    found = True
                    break
            
            if not found:
                missing_ingredients.append(f"{amount}g {ingredient}")
    
    result = {
        'total_calories': round(total_calories, 1),
        'total_protein': round(total_protein, 1),
        'total_carbs': round(total_carbs, 1),
        'total_fat': round(total_fat, 1),
        'ingredients': parsed_ingredients,
        'summary': f"Total: {round(total_calories)} kcal | {round(total_protein)}g protein | {round(total_carbs)}g carbs | {round(total_fat)}g fat"
    }
    
    if missing_ingredients:
        result['warning'] = f"⚠️  Could not find in database: {', '.join(missing_ingredients)}. Calculations may be incomplete."
        result['note'] = "The recipe generator's nutrition estimates are more comprehensive than the macro calculator's database."
    
    return result


def suggest_macro_adjustments(current_macros: dict, target_calories: int, target_protein: int) -> dict:
    """
    Suggest ingredient adjustments to hit macro targets.
    
    Args:
        current_macros: Current macro totals
        target_calories: Target calories
        target_protein: Target protein in grams
    
    Returns:
        Suggestions for adjustments
    """
    cal_diff = target_calories - current_macros['total_calories']
    protein_diff = target_protein - current_macros['total_protein']
    
    suggestions = []
    
    # Protein too low
    if protein_diff > 5:
        # Calculate amount of chicken breast needed (31g protein per 100g)
        chicken_needed = (protein_diff / 31) * 100
        # Calculate whey protein needed (80g protein per 100g)
        whey_needed = (protein_diff / 80) * 100
        suggestions.append(f"Add {round(chicken_needed)}g chicken breast or {round(whey_needed)}g whey protein to increase protein by {round(protein_diff)}g")
    
    # Protein too high
    elif protein_diff < -5:
        suggestions.append(f"Reduce protein source by approximately {round(abs(protein_diff)*3.5)}g to lower protein by {round(abs(protein_diff))}g")
    
    # Calories too low
    if cal_diff > 50:
        if protein_diff > 0:
            # Need both calories and protein - suggest lean protein
            chicken_for_cals = (cal_diff / 1.65)
            suggestions.append(f"Add {round(chicken_for_cals)}g chicken breast to add {round(cal_diff)} kcal and boost protein")
        else:
            # Just need calories - suggest carbs
            rice_needed = (cal_diff / 1.3)  # 130 cal per 100g rice
            suggestions.append(f"Add {round(rice_needed)}g rice or {round(cal_diff/3.89)}g oats to add {round(cal_diff)} kcal")
    
    # Calories too high
    elif cal_diff < -50:
        reduction_pct = (abs(cal_diff) / current_macros['total_calories']) * 100
        suggestions.append(f"Reduce all portions by approximately {round(reduction_pct)}% to lower calories by {round(abs(cal_diff))} kcal")
    
    if not suggestions:
        suggestions.append("✓ Macros are within acceptable range!")
    
    return {
        'calorie_difference': round(cal_diff, 1),
        'protein_difference': round(protein_diff, 1),
        'suggestions': suggestions,
        'percentage_off': {
            'calories': round((cal_diff / target_calories) * 100, 1) if target_calories > 0 else 0,
            'protein': round((protein_diff / target_protein) * 100, 1) if target_protein > 0 else 0
        }
    }

print("✓ Custom tools defined")

✓ Custom tools defined


## Define Agents

### Retry Options

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

### Define Each Agent

In [6]:
# Recipe Generator Agent
recipe_agent = Agent(
    name="RecipeGenerator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Expert at creating bodybuilding recipes based on nutritional goals",
    instruction="""You are a nutrition expert specializing in bodybuilding meal planning.

When users ask for recipes, generate them with:
- Calorie target (±50 kcal)
- Protein target (±5g)
- High-quality protein sources (chicken, beef, fish, eggs, dairy, legumes)
- Complex carbohydrates for energy
- Healthy fats in appropriate amounts
- Detailed instructions and nutrition facts

You have access to Google Search to:
- Find popular bodybuilding recipes matching the criteria
- Research cooking techniques
- Look up ingredient availability
- Check recipe ratings and reviews

Format your response as:

RECIPE NAME: [creative name]

INGREDIENTS:
- [ingredient 1 with exact amount in grams]
- [ingredient 2 with exact amount in grams]

INSTRUCTIONS:
1. [detailed step]
2. [detailed step]

NUTRITION (per serving):
- Calories: [amount] kcal
- Protein: [amount]g
- Carbs: [amount]g
- Fat: [amount]g

Make recipes practical, delicious, and optimized for muscle building!""",
    tools=[google_search, preload_memory],
    after_agent_callback=auto_save_to_memory
)

print("✓ Recipe Generator Agent created")

✓ Recipe Generator Agent created


In [7]:
# Substitution Agent
substitution_agent = Agent(
    name="SubstitutionExpert",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Expert at suggesting ingredient substitutions for bodybuilding recipes",
    instruction="""You are a nutrition expert specializing in ingredient substitutions.

When users ask about substitutions, provide 3-5 alternatives that:
- Maintain or improve nutritional value
- Prioritize protein content
- Consider dietary restrictions
- Include accurate conversion ratios

Format each substitution as:

SUBSTITUTION [number]:
Ingredient: [substitute name]
Ratio: [conversion, e.g., "1:1" or "use 1.5 cups instead of 1 cup"]
Impact: [how it affects taste, texture, cooking]
Nutrition: [protein, calories, carbs, fat comparison]
Bodybuilding Rating: [Better/Similar/Worse]

Focus on maintaining protein while keeping calories reasonable.""",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory
)

print("✓ Substitution Agent created")

✓ Substitution Agent created


In [8]:
# Shopping List Agent
shopping_list_agent = Agent(
    name="ShoppingListAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Expert at creating shopping lists, budget optimization, and meal prep planning",
    instruction="""You are an expert at meal planning and grocery shopping for bodybuilders.

You have access to Google Search to:
- Check current grocery prices
- Find deals and sales
- Compare store prices
- Research meal prep tips

For shopping lists, organize by category:

SHOPPING LIST

PRODUCE:
- [ ] [item] - [quantity]

PROTEIN:
- [ ] ⭐ [high-protein item] - [quantity]

DAIRY:
- [ ] [item] - [quantity]

GRAINS & CARBS:
- [ ] [item] - [quantity]

OTHER:
- [ ] [item] - [quantity]

ESTIMATED TOTAL COST: $[amount]

MEAL PREP TIPS:
- [tip 1]
- [tip 2]

For budget optimization:
- Search for current prices online
- Suggest bulk buying and budget-friendly protein sources
- Prioritize high-protein items
- Recommend seasonal alternatives

For meal prep schedules:
- Optimize preparation order for efficiency
- Identify batch cooking opportunities
- Provide storage and reheating instructions
- Include time estimates

Always combine duplicate ingredients and mark high-protein items with ⭐!""",
    tools=[google_search, preload_memory],
    after_agent_callback=auto_save_to_memory
)

print("✓ Shopping List Agent created")

✓ Shopping List Agent created


In [9]:
# Macro Calculator Agent
macro_calculator_agent = Agent(
    name="MacroCalculator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Expert at calculating and validating recipe macros with precision",
    instruction="""You are a nutrition calculator. You MUST use tools to calculate macros.

DO NOT answer from memory. DO NOT estimate. DO NOT guess.

When you receive a macro calculation request:

Step 1: Extract the ingredients list from the user's message
Step 2: IMMEDIATELY call calculate_recipe_macros() with the ingredients text
Step 3: Wait for the tool result
Step 4: Report the exact numbers from the tool result

Example:
User: "Calculate macros for 150g chicken, 200g rice"
You: [Call calculate_recipe_macros with "150g chicken\\n200g rice"]
Tool returns: {{'total_calories': 450, 'total_protein': 55, 'total_carbs': 56, 'total_fat': 6}}
You: "Here are the macros: 450 kcal, 55g protein, 56g carbs, 6g fat"

NEVER say things like "based on my knowledge" or "approximately" - ALWAYS use the tool.

If the tool returns a warning about missing ingredients, explain that to the user and show what was calculated.

Your tools:
- calculate_recipe_macros(ingredients_text) - REQUIRED for all calculations
- suggest_macro_adjustments(current_macros, target_calories, target_protein) - use when adjustments needed.""",
    tools=[calculate_recipe_macros, suggest_macro_adjustments, preload_memory],
    after_agent_callback=auto_save_to_memory
)


print("✓ Macro Calculator Agent created")

✓ Macro Calculator Agent created


### Plugins

In [10]:
import logging
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.plugins.base_plugin import BasePlugin
from google.adk.plugins.logging_plugin import LoggingPlugin
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Dict, Any


# Applies to all agent and model calls
class CustomPlugin(BasePlugin):
    """A Plugin that counts agent and tool invocations."""

    def __init__(self) -> None:
        """Initialize the plugin with counters."""
        super().__init__(name="count_invocation")
        self.agent_count: int = 0
        self.tool_count: int = 0
        self.llm_request_count: int = 0

    # Callback 1: Runs before an agent is called.
    async def before_agent_callback(
        self, *, agent: BaseAgent, callback_context: CallbackContext
    ) -> None:
        """Count agent runs."""
        self.agent_count += 1
        print(f"[Plugin] Agent called: {agent.name}")
        print(f"[Plugin] Agent run count: {self.agent_count}")

    # Callback 2: Runs before a model is called.
    async def before_model_callback(
        self, *, callback_context: CallbackContext, llm_request: LlmRequest
    ) -> None:
        """Count LLM requests."""
        self.llm_request_count += 1
        print(f"[Plugin] LLM request made. LLM request count: {self.llm_request_count}")

    # Callback 3: Runs before a tool is called.
    async def before_tool_callback(
        self, *, tool: BaseTool, tool_args: Dict[str, Any], tool_context: ToolContext
    ) -> None:
        """Count tool calls."""
        self.tool_count += 1
        print(f"[Plugin] Tool called: {tool.name}")
        print(f"[Plugin] Tool run count: {self.tool_count}")

## Create Orchestrator and Runner

In [11]:
# Wrap specialized agents as tools
recipe_tool = AgentTool(agent=recipe_agent)
macro_tool = AgentTool(agent=macro_calculator_agent)
substitution_tool = AgentTool(agent=substitution_agent)
shopping_tool = AgentTool(agent=shopping_list_agent)

In [12]:
# Create orchestrator agent
orchestrator = Agent(
    name="MacroForgeAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="Main orchestrator for the Recipe Adaptation Engine",
    instruction="""You are the main coordinator for a bodybuilding recipe adaptation system.

You have access to four specialized agents:

1. **RecipeGenerator**: Creates recipes (has Google Search)
2. **MacroCalculator**: Calculates and validates exact macros
3. **SubstitutionExpert**: Suggests ingredient substitutions
4. **ShoppingListAgent**: Creates shopping lists, optimizes budgets, plans meal prep (has Google Search)

Your job:
1. Understand what the user needs
2. Delegate to the appropriate agent(s)
3. Coordinate multi-step workflows
4. Present results clearly and helpfully

IMPORTANT WORKFLOW RULES:

When user asks to "create a recipe" or "generate a recipe":
1. FIRST: Call RecipeGenerator to create the recipe
2. Show the user the complete recipe from RecipeGenerator
3. THEN: Optionally call MacroCalculator to validate (only if needed)
4. Present both results to the user

When user asks to "calculate macros" for an existing recipe:
1. ONLY call MacroCalculator (no need for RecipeGenerator)

When user asks for "substitutions":
1. ONLY call SubstitutionExpert

When user asks for "shopping list" or "meal prep":
1. Call ShoppingListAgent

Common patterns:
- "Create a recipe with X calories and Y protein" → RecipeGenerator FIRST (show recipe), then optionally MacroCalculator
- "Calculate macros for this recipe" → MacroCalculator ONLY
- "Make a meal plan" → RecipeGenerator (multiple times for each meal), show all recipes
- "Substitute [ingredient]" → SubstitutionExpert
- "Shopping list" → ShoppingListAgent (can search for prices)
- "Meal prep schedule" → ShoppingListAgent

Best practices:
- ALWAYS show the recipe to the user before doing validation
- Don't skip steps - users want to see the creative output first
- When validating with MacroCalculator, present it as additional verification, not replacement
- Use agents' search capabilities when pricing or recipe research is needed
- Chain agents for complete workflows (recipe → macro check → shopping list)

Be conversational and helpful!""",
    tools=[recipe_tool, macro_tool, substitution_tool, shopping_tool, preload_memory],
    after_agent_callback=auto_save_to_memory
)

In [13]:
# Create Runner
session_service = InMemorySessionService() 
memory_service = InMemoryMemoryService()

# Runner with session_service and memory_service
runner = Runner(agent=orchestrator, 
                app_name="MacroForgeApp",
                session_service=session_service,
                memory_service=memory_service
                )

# InMemoryRunner with plugins
runner_with_plugins = InMemoryRunner(agent=orchestrator,
                        plugins=[CustomPlugin()]
                       )

print("✓ Runner created")
print("\n🏋️  Recipe Adaptation Engine Ready!")
print("Ask me anything about recipes, meal planning, substitutions, or shopping lists.")

✓ Runner created

🏋️  Recipe Adaptation Engine Ready!
Ask me anything about recipes, meal planning, substitutions, or shopping lists.


## Test the agents

#### Test runner for different scenarios

In [14]:
# Scenario 1: Create a recipe with macro goals
question = """
I need help with my meal prep:
Create a 500 calorie breakfast with 30g protein.
"""
response = await runner.run_debug(question)


 ### Created new session: debug_session_id

User > 
I need help with my meal prep:
Create a 500 calorie breakfast with 30g protein.



MacroForgeAgent > This breakfast recipe sounds delicious and meets your macro goals:

**Savory Cottage Cheese & Smoked Salmon Scramble**

This quick and easy breakfast is packed with protein and healthy fats to fuel your morning.

INGREDIENTS:
- Low-fat cottage cheese: 150g
- Smoked salmon: 50g
- Large eggs: 2 (approx. 100g)
- Spinach: 30g
- Whole grain toast: 1 slice (approx. 30g)
- Black pepper: to taste

INSTRUCTIONS:
1. In a non-stick skillet, lightly sauté the spinach until wilted.
2. Add the cottage cheese and smoked salmon to the skillet. Stir gently to combine.
3. Whisk the eggs in a small bowl with black pepper. Pour the eggs over the cottage cheese and salmon mixture.
4. Scramble until the eggs are cooked through.
5. Serve immediately with a slice of whole grain toast.

NUTRITION (per serving):
- Calories: 495 kcal
- Protein: 31g
- Carbs: 25g
- Fat: 26g

Would you like me to calculate the macros for this recipe to validate them, or would you like to proceed with another reque

In [15]:
# Scenario 2: Calculate macros
question = """
I have this recipe:
- 150g chicken breast
- 200g sweet potato
- 50g spinach
Calculate the macros
"""
response = await runner.run_debug(question)


 ### Continue session: debug_session_id

User > 
I have this recipe:
- 150g chicken breast
- 200g sweet potato
- 50g spinach
Calculate the macros



MacroForgeAgent > Here are the macros for your recipe: 431 kcal, 51.2g protein, 41.8g carbs, 5.8g fat.


In [16]:
# Scenario 3: Create shopping list
question = """
Create a shopping list for:
1. Grilled chicken with rice and broccoli (3 servings)
"""
response = await runner.run_debug(question)


 ### Continue session: debug_session_id

User > 
Create a shopping list for:
1. Grilled chicken with rice and broccoli (3 servings)



MacroForgeAgent > Here is your shopping list for 3 servings of grilled chicken with rice and broccoli:

**SHOPPING LIST**

**PRODUCE:**
*   Broccoli Crowns - 1.5 lbs

**PROTEIN:**
*   Boneless, Skinless Chicken Breasts - 1.5 lbs

**GRAINS & CARBS:**
*   White Rice (Long Grain) - 1.5 cups (uncooked)

**OTHER:**
*   Olive Oil - 1 tablespoon (for cooking)
*   Salt - to taste
*   Black Pepper - to taste
*   Garlic Powder - 1 teaspoon (optional, for seasoning chicken)
*   Paprika - 1/2 teaspoon (optional, for seasoning chicken)
*   Cumin - 1/2 teaspoon (optional, for seasoning chicken)

**MEAL PREP TIPS:**

*   **Batch Cook Rice:** Cook the entire 1.5 cups of rice at once. Once cooled, portion it into individual containers.
*   **Grill/Bake Chicken in Advance:** Grill or bake all 1.5 lbs of chicken breasts. Let them cool completely before slicing or cubing for easier portioning. This ensures a quick assembly later in the week.
*   **Steam or Roast Broccoli:** Broccoli can be steamed or roas

In [17]:
# Scenario 4: Substitution
question = """
Substitute chicken with beef in this recipe for similar protein content
- 150g chicken thigh
- 200g white rice
- 50g broccoli
"""
response = await runner.run_debug(question)


 ### Continue session: debug_session_id

User > 
Substitute chicken with beef in this recipe for similar protein content
- 150g chicken thigh
- 200g white rice
- 50g broccoli



MacroForgeAgent > Here are a few beef substitutions for your recipe, keeping similar protein content in mind:

**SUBSTITUTION 1:**
*   **Ingredient:** Lean Ground Beef (90% lean)
*   **Ratio:** 1:1 (150g ground beef for 150g chicken thigh)
*   **Impact:** Ground beef will have a slightly coarser texture than chicken thigh and will likely require draining excess fat after cooking. Flavor profile will be richer and beefier.
*   **Nutrition:** Similar protein content to chicken thigh. Lean ground beef will have slightly more fat and calories than chicken thigh.
*   **Bodybuilding Rating:** Similar

**SUBSTITUTION 2:**
*   **Ingredient:** Sirloin Steak
*   **Ratio:** 1:1 (150g sirloin steak for 150g chicken thigh)
*   **Impact:** Sirloin steak will offer a firmer, chewier texture. Requires cubing or slicing and cooking methods like stir-frying or pan-searing. Flavor is distinctly beefy.
*   **Nutrition:** Leaner than chicken thigh, offering comparable protein with potentially lower fat and

#### Test runner with plugin

In [18]:
question = """
I need help with my meal prep:
Create a 500 calorie breakfast with 30g protein.
"""
response = await runner_with_plugins.run_debug(question)


 ### Created new session: debug_session_id

User > 
I need help with my meal prep:
Create a 500 calorie breakfast with 30g protein.

[Plugin] Agent called: MacroForgeAgent
[Plugin] Agent run count: 1
[Plugin] LLM request made. LLM request count: 1


[Plugin] Tool called: RecipeGenerator
[Plugin] Tool run count: 1
[Plugin] Agent called: RecipeGenerator
[Plugin] Agent run count: 2
[Plugin] LLM request made. LLM request count: 2
[Plugin] LLM request made. LLM request count: 3
MacroForgeAgent > Here's a delicious and protein-packed breakfast to get your day started right!

## Muscle Fuel Scramble

This hearty scramble is packed with protein and complex carbohydrates to fuel your day and support muscle growth.

**INGREDIENTS:**
- 200g egg whites
- 100g lean ground turkey
- 50g rolled oats
- 100g mixed berries (e.g., blueberries, raspberries)
- 10g chia seeds
- 5g olive oil
- Salt and pepper to taste

**INSTRUCTIONS:**
1. Cook the rolled oats according to package directions. You can add water or unsweetened almond milk.
2. While the oats are cooking, heat the olive oil in a non-stick skillet over medium heat. Add the lean ground turkey and cook, breaking it up with a spoon, until browned and cooked through.
3. Add the egg whites to the 

#### Test run_session

In [19]:
question = """
Create an 800-calorie lunch with 60g of protein
"""

response = await run_session(runner, question, "test-session")


### Session: test-session

User > 
Create an 800-calorie lunch with 60g of protein



Model: > The macros for the Hearty Chicken and Sweet Potato Power Bowl are:

*   **Calories:** 795 kcal
*   **Protein:** 62g
*   **Carbohydrates:** 60g
*   **Fat:** 29g

Please note that the MacroCalculator was unable to find some ingredients, so the calculated macros might not be entirely accurate.

Would you like me to suggest any substitutions for the ingredients that were not found, or would you like to proceed with anything else?


### Test that shared memory works

In [20]:
await run_session(runner, "My favorite protein is chicken", "test-session-1")


### Session: test-session-1

User > My favorite protein is chicken
Model: > Great! Chicken is a fantastic source of lean protein.

Are you looking to create a new recipe, calculate macros for an existing one, find substitutions, or create a shopping list? Let me know how I can help you with your chicken-based meals!


In [21]:
await run_session(runner, "What's my favorite protein?", "test-session-2")


### Session: test-session-2

User > What's my favorite protein?
Model: > Your favorite protein is chicken. Did you want to create a new recipe, calculate macros, find substitutions, or create a shopping list today?


### Interactive Session

#### Interactive chat - User Input Version
Commented as it's not compatible with Kaggle

In [22]:
# # Interactive chat 
# print("\n🏋️  Recipe Adaptation Engine - Interactive Mode")
# print("Type 'q', 'quit', or 'exit' to stop\n")

# while True:
#     user_input = input("You: ").strip()
    
#     if user_input.lower() in ['quit', 'exit', 'q']:
#         print("\nGoodbye! 💪\n")
#         break
    
#     if not user_input:
#         continue
    
#     print(f"\n{'='*70}")
#     response = await run_session(runner, user_input, "interative-session")
#     print(f"{'='*70}\n")

#### Interactive chat - Kaggle Compatible Version

In [23]:
# Interactive chat - Kaggle Compatible Version
# Since Kaggle doesn't support input(), define your queries in a list

print("\n🏋️  Recipe Adaptation Engine - Batch Mode (Kaggle Compatible)")
print("="*70)
print("\nDefine your queries in the 'queries' list below and run the cell.\n")

# Add your queries here
queries = [
    "Create a recipe with 800 calories and 60 grams of protein",
    "Calculate the macros for this recipe",
    "Substitute chicken with ground beef",
    "Go with 90% lean ground beef and create a shopping list for 2 servings"
]

# Process all queries
for i, query in enumerate(queries, 1):
    print(f"\n{'='*70}")
    print(f"Query {i}/{len(queries)}: {query}")
    print(f"{'='*70}\n")
    
    response = await run_session(runner, query, "interative-session")
    print()

print("\n" + "="*70)
print("✓ All queries completed!")
print("="*70)
print("\nTo add more queries, modify the 'queries' list above and re-run this cell.")


🏋️  Recipe Adaptation Engine - Batch Mode (Kaggle Compatible)

Define your queries in the 'queries' list below and run the cell.


Query 1/4: Create a recipe with 800 calories and 60 grams of protein


### Session: interative-session

User > Create a recipe with 800 calories and 60 grams of protein


Model: > Here is a recipe for **Spicy Garlic Butter Salmon with Roasted Asparagus and Quinoa**:

**Spicy Garlic Butter Salmon with Roasted Asparagus and Quinoa**

This recipe is designed to be a nutrient-dense meal, perfect for supporting muscle growth and recovery. It features a high-quality protein source from salmon, complex carbohydrates from quinoa, and healthy fats to aid in nutrient absorption and hormone production.

**INGREDIENTS:**

*   225g Salmon fillet
*   15g Olive oil
*   15g Butter
*   3 cloves Garlic, minced
*   1g Red pepper flakes (adjust to taste)
*   15ml Lemon juice
*   200g Asparagus, trimmed
*   100g Quinoa, dry
*   200ml Water or vegetable broth
*   Salt, to taste
*   Black pepper, to taste

**INSTRUCTIONS:**

1.  **Prepare Quinoa:** Rinse the quinoa thoroughly under cold water. In a small saucepan, combine the rinsed quinoa with water or vegetable broth, and a pinch of salt. Bring to a boil, then reduce heat to low, cover, and simmer for 15-20 minutes, or unti

Model: > I can calculate the macros for the entire recipe, but I need the complete list of ingredients with their exact quantities. The tool was only able to provide macros for the salmon fillet.

Could you please provide the full ingredients list again so I can get you accurate macro calculations?


Query 3/4: Substitute chicken with ground beef


### Session: interative-session

User > Substitute chicken with ground beef


Model: > You mentioned substituting chicken with ground beef, but the recipe we were discussing featured salmon, not chicken.

If you'd like to substitute the **225g Salmon Fillet** in the **Spicy Garlic Butter Salmon with Roasted Asparagus and Quinoa** recipe with ground beef, here are a few options:

**1. Lean Ground Beef (90% lean / 10% fat)**
*   **Ratio:** Use 225g of ground beef.
*   **Impact:** This will significantly change the dish's texture from flaky salmon to crumbly beef. The flavor will shift from oceanic to savory. You'd likely need to brown the beef separately and adjust how it's incorporated. The garlic butter sauce will also pair differently.
*   **Nutrition:** Approx. 450-500 kcal, 45-50g protein, 0g carbs, 25-30g fat.
*   **Bodybuilding Rating:** Similar protein, but higher in saturated fat.

**2. Extra Lean Ground Beef (93% lean / 7% fat)**
*   **Ratio:** Use 225g of ground beef.
*   **Impact:** Similar to lean ground beef, but with a slightly leaner profile. Brown

Model: > Here is your shopping list for 2 servings of the adapted recipe, using 90% lean ground beef instead of salmon. I've included all the other ingredients from the original recipe as well.

**Estimated Cost Note:** Prices for ground beef can vary by location and store. The estimate below is based on an average price for 90% lean ground beef.

## SHOPPING LIST

**PRODUCE:**
*   [ ] Garlic - 4 cloves
*   [ ] Lemon - 1 whole
*   [ ] Asparagus - 1 lb

**PROTEIN:**
*   [ ] ⭐ 90% Lean Ground Beef - 1 lb (454g)

**DAIRY:**
*   [ ] Salted Butter - 2 tbsp

**GRAINS & CARBS:**
*   [ ] Quinoa - 1/2 cup

**OTHER:**
*   [ ] Olive Oil - 2 tbsp (plus more for cooking)
*   [ ] Salt - to taste
*   [ ] Black Pepper - to taste
*   [ ] Water - 1 cup (for quinoa)
*   [ ] Vegetable Broth - 1/4 cup (optional, for quinoa, for extra flavor)

**ESTIMATED TOTAL COST:** $7.50 (This estimate is primarily for the ground beef, as other ingredients are likely already stocked or are very inexpensive.)

## MEAL PR

### If I had more time: Deploy to Vertex AI Agent Engine